# Week 4 Tasks
Connect your Pi Pico or ESP32 to your computer via Jupyter.

### Configure your serial connection to your microcontroller. Use the following commands to establish a serial connection.
#### For Windows
 %serialconnect to --port=COM3 --baud=115200
#### For macOS
 %serialconnect to --port=/dev/tty.SLAB_USBtoUART --baud=115200

In [78]:
%serialconnect to --port=COM8 --baud=115200

Connecting to --port=COM8 --baud=115200 
MicroPython v1.24.1 on 2024-11-29; Generic ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


## Task 1: Trafic light Prototype
### Scenario
A small municipality has hired you to prototype a simple traffic light controller for a quiet residential intersection. The local council wants an easy-to-follow Red-Yellow-Green cycle to regulate car traffic. Your job is to demonstrate a working model of the traffic light using a Raspberry Pi Pico (or any microcontroller) and three LEDs.

### Key Requirements
##### Traffic Light Cycle
 1. Red → Yellow → Green
 2. Red ON (enough time to ensure vehicles fully stop).
 3. Yellow ON (brief warning).
 4. Green ON (allow traffic flow).
 5. Cycle repeats continuously, with no overlap of multiple lights.
### Hardware Requriment
 1. Breadboard
 2. Three LED lights (Green, Yellow, Red)
 3. Pi Pico or ESP32
 4. Jumper wires 


In [18]:
import machine
import time

red = machine.Pin(13, machine.Pin.OUT)
yellow = machine.Pin(12, machine.Pin.OUT)
green = machine.Pin(14, machine.Pin.OUT)

while True:
    green.on()
    time.sleep(15)
    
    green.off()
    yellow.on()
    time.sleep(3)
    
    yellow.off()
    red.on()
    time.sleep(15)
    red.off()


....

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 11, in <module>
KeyboardInterrupt: 


## Task 2: Basic Traffic light with Pedestrian Button and light
### Scenario
After successfully demonstrating a basic traffic light prototype, the city council of “Greenfield Heights” has requested an enhanced version to accommodate pedestrians at a busy section of Main Street safely. 
Specifically, they want:

1.	Retain the existing traffic light cycle (Red, Yellow, Green) for vehicles.
2.	Add a pedestrian signal with Green (Walk) and Red (Don’t Walk) lights.
3.	Include a pedestrian button so pedestrians can request to cross.
### Hardware Requirement
 1. Breadbord
 2. Five LED lights (Green, Yellow, Red, for vehicles. Green and Red for Predistrain)
 3. Button
 4. Pi Pico or ESP32
 5. Jumper wires 


In [68]:
import machine
import time
import _thread
import math

cycle_time = 18

# Setup for traffic lights and pedestrian lights
red = machine.Pin(13, machine.Pin.OUT)
yellow = machine.Pin(12, machine.Pin.OUT)
green = machine.Pin(14, machine.Pin.OUT)

p_red = machine.Pin(25, machine.Pin.OUT)
p_green = machine.Pin(33, machine.Pin.OUT)
p_button = machine.Pin(26, machine.Pin.IN, machine.Pin.PULL_UP)

# Setup for buzzer
buzzer = machine.PWM(machine.Pin(32))
buzzer.freq(1)  # Set frequency in Hz
buzzer.duty(0)

p_request = False

# Reset all lights
p_red.on()
p_green.off()
red.off()
yellow.off()
green.off()

# Helper function to play tone
def play_tone(frequency, duration):
    buzzer.freq(frequency)  # Set frequency in Hz
    buzzer.duty(512)        # Set duty cycle (range: 0-1023)
    time.sleep(duration)    # Play tone for specified duration
    buzzer.duty(0)

# Green crossing sound (two-tone rapid ticking)
def green_crossing_sound():
    for _ in range(56):  # Adjust loop for total duration
        play_tone(3000, 0.05)  # First tone: 3000 Hz for 0.05 seconds
        time.sleep(0.01)       # Small delay between tones
        play_tone(3500, 0.05)  # Second tone: 3500 Hz for 0.05 seconds
        time.sleep(0.1)        # Delay before the next "tick-tick"

# Flashing red sound (slower beeping)
def red_signal_sound():
    for _ in range(3):  # Adjust loop for total duration
        play_tone(1000, 0.5)  # 1000 Hz tone for 0.5 seconds
        time.sleep(0.5)       # Gap between beeps

# Monitor the pedestrian button
def monitor_button():
    global p_request
    while True:
        if p_button.value() == 0:  # Button pressed
            p_request = True
            time.sleep(0.5)  # Small delay to debounce

# Defines the main traffic light cycle
def traffic_cycle():
    green.on()  # Start with green light for vehicles
    time.sleep(cycle_time)
    green.off()
    yellow.on()  # Amber light warning
    time.sleep(3)
    yellow.off()
    red.on()  # Red light for vehicles

    if p_request:
        time.sleep(2)  # Small delay before pedestrians can cross
        p_cycle()  # Run pedestrian cycle
        time.sleep(1)  # Short safety delay
    else:
        time.sleep(cycle_time)

    red.off()  # Turn off red light ready for restart

# Defines the pedestrian light cycle
def p_cycle():
    global p_request

    # Green light for pedestrians
    p_red.off()
    p_green.on()

    # Run green crossing sound in a separate thread
    _thread.start_new_thread(green_crossing_sound, ())

    time.sleep(cycle_time - (cycle_time / 4) - 1)

    # Flashing red light for pedestrians
    p_green.off()
    for _ in range(math.floor((cycle_time / 4) - 1)):
        p_red.on()

        # Run red signal sound in a separate thread
        _thread.start_new_thread(red_signal_sound, ())

        time.sleep(0.5)
        p_red.off()
        time.sleep(0.5)

    p_red.on()  # Final steady red
    p_request = False  # Reset pedestrian request

# Start button monitoring in a separate thread
_thread.start_new_thread(monitor_button, ())

# Main loop
while True:
    traffic_cycle()  # Run the traffic cycle


..........Pedestrian requested crossing
Pedestrian requested crossing
Pedestrian requested crossing
Pedestrian requested crossing
PedestrPePeian requested crossing
Pedestrian requested crossding
estrian requested crossing
destrian requested crossing
Pedestrian requested crossingPedestrian requested crossi
Pedng
estrian requested crossing
Pedestrian requested crossingPedestrian requested crossing

Pedestrian requested crPeossing
destrian requested crossing
Pedestrian requested crossingPedestrian requested crossing

Pedestrian reqPeuested crossing
destrian requested crossing
Pedestrian requested crossingPedestrian requested crossing

PedestrPeian requested crossing
destrian requested crossing
Pedestrian requested crossingPedestrian requested crossi
Peng
destrian requested crossing
Pedestrian requested crossingPedestrian requested crossing

Pedestrian requested cPerossing
destrian requested crossing
Pedestrian requested crossingPedestrian requested crossing

Pedestrian rePequested crossin

Traceback (most recent call last):
  File "<stdin>", line 114, in <module>
  File "<stdin>", line 77, in traffic_cycle
KeyboardInterrupt: 


## Task 3: Water Tank Level Indicator System
### Scenario
Greenfield Municipality requires a visual representation of the water level in its small-scale reservoir tank. You've been asked to create a simplified IoT prototype using LEDs to indicate the current state of the reservoir, simulating both filling (ascending) and draining (descending) sequences.
### Operational Description:
A sequence of LEDs represents the water level inside the tower. The LEDs will light sequentially from the bottom to the top, indicating the tank is filling, and then in reverse direction, indicating draining. This visual feedback helps technicians and municipality staff quickly assess tank operations and ensure proper functionality.

Additionally, each LED activation status (e.g., "Filling: LED 1 activated", "Draining: LED 4 activated") should be printed to the serial console for remote monitoring.
### Hardware Requriment
 1. Breadbord
 2. Four LED lights (Preferably different colours for clarity)
 3. Pi Pico or ESP32
 4. Jumper wires 

#### Key Requirements: 
To meet the municipality’s coding standards, you must:

1.	Write a function that controls the ascending/descending LED logic.
2.	Inside this function, use at least one for loop to iterate through your list of LEDs in ascending and descending order.



In [81]:
from machine import Pin
from time import sleep

red = Pin(13, Pin.OUT)
yellow = Pin(12, Pin.OUT)
green1 = Pin(14, Pin.OUT)
green2 = Pin(27, Pin.OUT)
delay = 0.5

def reset():
    red.off()
    yellow.off()
    green1.off()
    green2.off()

def fill_tank():
    green2.on()
    print("Filling: LED 1 activated")
    sleep(delay)
    green1.on()
    print("Filling: LED 2 activated")
    sleep(delay)
    yellow.on()
    print("Filling: LED 3 activated")
    sleep(delay)
    red.on()
    print("Filling: LED 4 activated")
    sleep(delay)

def drain_tank():
    red.off()
    print("Draining: LED 4 deactivated")
    sleep(delay)
    yellow.off()
    print("Draining: LED 3 deactivated")
    sleep(delay)
    green1.off()
    print("Draining: LED 2 deactivated")
    sleep(delay)
    green2.off()
    print("Draining: LED 1 deactivated")
    sleep(delay)

while True:
    reset()
    fill_tank()
    sleep(1)
    drain_tank()
    sleep(1)

.....

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 40, in <module>
  File "<stdin>", line 26, in fill_tank
KeyboardInterrupt: 


## Task 4: Multi-Mode LED Display for Community Events
### Scenario
The Greenfield Heights Recreation Department wants a multi-purpose LED display to be used at various community events—fairs, holiday gatherings, and educational demos. 
They’ve already seen a basic LED ladder prototype but now request multiple behaviour patterns (modes) that a user can select:

1.	Mode A: Basic Fill & Drain – LEDs turn on from bottom to top, then off from top to bottom.
2.	Mode B: Reverse – LEDs turn on from bottom to top, then back down.
### Hardware Requriment
 1. Breadbord
 2. LED lights
 3. Two Buttons
 4. Pi Pico or ESP32     
 5. Jumper wires 


In [82]:
from machine import Pin
from time import sleep

red = Pin(13, Pin.OUT)
yellow = Pin(12, Pin.OUT)
green1 = Pin(14, Pin.OUT)
green2 = Pin(27, Pin.OUT)

button = Pin(26, Pin.IN, Pin.PULL_UP)
button2 = Pin(25, Pin.IN, Pin.PULL_UP)

delay = 0.5

def reset():
    red.off()
    yellow.off()
    green1.off()
    green2.off()

def fill_tank():
    green2.on()
    print("Filling: LED 1 activated")
    sleep(delay)
    green1.on()
    print("Filling: LED 2 activated")
    sleep(delay)
    yellow.on()
    print("Filling: LED 3 activated")
    sleep(delay)
    red.on()
    print("Filling: LED 4 activated")
    sleep(delay)

def drain_tank():
    red.off()
    print("Draining: LED 4 deactivated")
    sleep(delay)
    yellow.off()
    print("Draining: LED 3 deactivated")
    sleep(delay)
    green1.off()
    print("Draining: LED 2 deactivated")
    sleep(delay)
    green2.off()
    print("Draining: LED 1 deactivated")
    sleep(delay)

reset()

while True:
    if (button.value() == 0):
        drain_tank()
    if(button2.value() == 0):
        fill_tank()

Filling: LED 1 activated
Filling: LED 2 activated
Filling: LED 3 activated
Filling: LED 4 activated
Draining: LED 4 deactivated
Draining: LED 3 deactivated
Draining: LED 2 deactivated
Draining: LED 1 deactivated
.Filling: LED 1 activated
Filling: LED 2 activated
Filling: LED 3 activated
Filling: LED 4 activated
Draining: LED 4 deactivated
Draining: LED 3 deactivated
Draining: LED 2 deactivated
Draining: LED 1 deactivated
Filling: LED 1 activated
Filling: LED 2 activated
Filling: LED 3 activated
Filling: LED 4 activated
.Draining: LED 4 deactivated
Draining: LED 3 deactivated
Draining: LED 2 deactivated
Draining: LED 1 deactivated
Filling: LED 1 activated
Filling: LED 2 activated
Filling: LED 3 activated
Filling: LED 4 activated
Draining: LED 4 deactivated
Draining: LED 3 deactivated
Draining: LED 2 deactivated
Draining: LED 1 deactivated
.......Filling: LED 1 activated
Filling: LED 2 activated
Filling: LED 3 activated
Filling: LED 4 activated
.Draining: LED 4 deactivated
Draining: LED 

Traceback (most recent call last):
  File "<stdin>", line 54, in <module>
KeyboardInterrupt: 


## Task 5: Data Centre Monitoring and Alert System
### Scenario
You are responsible for managing a small data centre that requires continuous monitoring. You've implemented a basic IoT-based alert system utilising an ESP32 or Raspberry Pi Pico microcontroller and LEDs. This system provides clear visual and textual indicators of system statuses to allow technicians to quickly identify and respond to scenarios such as emergencies, normal operation, cooling activities, or maintenance requirements.
#### LEDs Status Indicators:

 I. Red LED: Indicates an Emergency Alert (e.g., overheating, critical hardware failure).
    
 II. Green LED: Indicates Normal Operation (systems are running smoothly).
    
 III. Blue LED: Indicates System Cooling (active cooling processes in progress).
    
 IV. Yellow LED: Indicates Maintenance Required (scheduled or proactive maintenance is needed).
    
Additionally, the system logs these statuses by outputting messages to the JupyterLab console, providing visual and textual status updates.

### Hardware Requriment
 1. Breadbord
 2. LED lights (Red, Green, Blue and Yellow)
 3. Button
 4. Pi Pico or ESP32
 5. Jumper wires 



In [ ]:
from machine import Pin
from time import sleep

button = Pin(26, Pin.IN, Pin.PULL_UP)

delay = 0.1  # Delay for smoother operation
modes = [
    ['Red: Emergency!', Pin(13, Pin.OUT)],
    ['Green: Normal operation', Pin(27, Pin.OUT)],
    ['Blue: System cooling', Pin(14, Pin.OUT)],
    ['Yellow: Maintenance Required', Pin(12, Pin.OUT)]
]
current_mode = 0

def reset():
    for mode in modes:
        mode[1].off()

# Initialize button state
previous_state = 1  # Button not pressed (pulled up)

while True:
    current_state = button.value()  # Read the button state
    
    # Check if the button was released (transition from pressed to not pressed)
    if previous_state == 0 and current_state == 1:
        current_mode += 1  # Increment mode
        if current_mode >= len(modes):  # Cycle back to the first mode
            current_mode = 0
        print(modes[current_mode][0])
    
    # Update the lights
    for i in range(len(modes)):
        if i == current_mode:
            modes[i][1].on()
        else:
            modes[i][1].off()
    
    previous_state = current_state  # Update previous state
    sleep(delay)  # Short delay for debounce


......Green: Normal operation
.Blue: System cooling
.Yellow: Maintenance Required
.Red: Emergency!
.Green: Normal operation
Blue: System cooling
Yellow: Maintenance Required
Red: Emergency!
Green: Normal operation
Blue: System cooling
Yellow: Maintenance Required
Red: Emergency!
Green: Normal operation
Blue: System cooling
Yellow: Maintenance Required
...Red: Emergency!
Green: Normal operation
Blue: System cooling
.Yellow: Maintenance Required
Red: Emergency!
Green: Normal operation
Blue: System cooling
.Yellow: Maintenance Required
.Red: Emergency!
....Green: Normal operation
.Blue: System cooling
....Yellow: Maintenance Required
.......Red: Emergency!
Green: Normal operation
Blue: System cooling
Yellow: Maintenance Required
Red: Emergency!
Green: Normal operation
Blue: System cooling
Yellow: Maintenance Required
Red: Emergency!
Green: Normal operation
Blue: System cooling
Yellow: Maintenance Required
Red: Emergency!
Green: Normal operation
Blue: System cooling
Yellow: Maintenance Req